In [21]:
import numpy as np
from scipy.linalg import solve_sylvester, norm
import scipy.sparse as sp

In [22]:
def newton_nare(A, B, C, D, X0, tol=1e-13, kmax=30):
    """
    Newton's method for solving the Nonlinear Algebraic Riccati Equation (NARE):
    C + XA + DX - XBX = 0
    """
    X = X0.copy()
    k = 0
    err = 1

    while err > tol and k < kmax:
        # Compute residual RX = C + XA + DX - XBX
        RX = C + X @ A + D @ X - X @ B @ X

        # Solve the Sylvester equation (D - XB)H + H(A - BX) = -RX for H
        H = solve_sylvester(D - X @ B, A - B @ X, -RX)

        # Update X
        X = X + H

        # Calculate the error; changed from l1 to frobenius
        err = norm(H, 1) / norm(X, 1)
        # err = norm(H, 'fro') / (1 + norm(X, 'fro'))

        if k % 5 == 0:  # Print every 5 iterations
            print(f"Iteration {k}, Error: {err:.2e}")
        
        # Increment iteration counter
        k += 1

    # Check if the solution converged
    if k == kmax:
        print("Warning: reached the maximum number of iterations without convergence.")

    return X

In [23]:
def ensure_positive_definite(M, epsilon=1e-3, min_threshold=1e-3):
    """Ensure M is well-conditioned ; add small value to diagonal if needed"""
    min_eig = np.min(np.linalg.eigvals(M))
    
    if min_eig < min_threshold:
        print(f"Minimum eigenvalue too small ({min_eig:.2e}), adding {epsilon} to diagonal elements.")
        M += np.eye(M.shape[0]) * (abs(min_eig) + epsilon)
    
    return M

In [24]:
def make_sparse_spd_matrix(
    n_dim=10,
    alpha=0.95,
    norm_diag=True,
    smallest_coef=0.1,
    largest_coef=0.9,
    random_state=42
):
    """
    Generate a sparse symmetric positive definite (SPD) matrix.

    Parameters
    ----------
    n_dim : int, default=10
        The size of the random matrix to generate.

    alpha : float, default=0.95
        The probability that a coefficient is zero, controlling sparsity.
        Higher values mean more sparsity. Should be between 0 and 1.

    norm_diag : bool, default=False
        If True, normalizes the matrix so that the diagonal elements are all 1.

    smallest_coef : float, default=0.1
        The smallest coefficient in the randomly generated values (between 0 and 1).

    largest_coef : float, default=0.9
        The largest coefficient in the randomly generated values (between 0 and 1).

    random_state : int or None, default=None
        Seed for random number generation, ensuring reproducible results.

    Returns
    -------
    ndarray or sparse matrix
        The generated sparse SPD matrix as a dense ndarray.
    """
    rng = np.random.default_rng(random_state)

    # Start with a negative identity matrix, which will form the basis of the Cholesky factor.
    chol = -sp.eye(n_dim, format="csc")

    # Generate a random sparse lower triangular matrix to add sparsity
    aux = sp.random(
        m=n_dim, n=n_dim, density=1 - alpha,
        data_rvs=lambda x: rng.uniform(low=smallest_coef, high=largest_coef, size=x),
        format="csc"
    )
    aux = sp.tril(aux, k=-1, format="csc")

    # Randomly permute rows and columns to avoid asymmetries
    permutation = rng.permutation(n_dim)
    aux = aux[permutation].T[permutation]

    # Add the sparse auxiliary matrix to the Cholesky factor
    chol += aux

    # Form the SPD matrix by taking the product of the Cholesky factor with its transpose
    prec = chol.T @ chol

    # Optionally normalize the diagonal to 1
    if norm_diag:
        d = sp.diags(1.0 / np.sqrt(prec.diagonal()))
        prec = d @ prec @ d
    prec = ensure_positive_definite(prec.toarray())

    return prec

In [25]:
# random dense matrices

n=10

A = np.zeros((n, n))

B = np.random.rand(n, n)
# symmetric positive definite
B = B.T @ B + np.eye(n)  
min_eig = np.min(np.linalg.eigvals(B))
print(f"Minimum eigenvalue ({min_eig:.2e})")

C = -2*np.eye(n)

# symmetric D
D = np.random.rand(n, n)
D = D.T @ D 

# initial guess
X0 = np.eye(n)

X_hat = newton_nare(A, B, C, D, X0)

residual = C + X_hat @ A + D @ X_hat - X_hat @ B @ X_hat
residual_norm = norm(residual, 'fro')

print('Residual after NARE: ', residual)
print('Its norm: ', residual_norm)

Minimum eigenvalue (1.02e+00)
Iteration 0, Error: 1.36e+00
Iteration 5, Error: 9.82e-01
Iteration 10, Error: 1.00e+00
Iteration 15, Error: 4.12e-01
Iteration 20, Error: 8.29e-01
Iteration 25, Error: 1.11e-01
Residual after NARE:  [[ 1.51283362e+06  3.88149184e+06 -3.97708957e+06  1.03458230e+06
  -3.61300715e+06 -3.01498520e+06 -1.39675378e+06 -2.03259415e+05
  -1.12804727e+06  7.10440876e+06]
 [ 1.31649800e+06  3.40167016e+06 -3.48182435e+06  6.92578679e+05
  -3.03400981e+06 -2.30237555e+06 -9.41564973e+05 -2.77362296e+05
  -1.01222275e+06  5.71161852e+06]
 [ 8.75450925e+05  2.24610001e+06 -2.29839329e+06  5.32777124e+05
  -2.04849798e+06 -1.63569701e+06 -7.09351298e+05 -1.57029395e+05
  -6.59195991e+05  3.93803513e+06]
 [ 4.38194620e+05  1.11260519e+06 -1.14160808e+06  3.98556945e+05
  -1.09878454e+06 -1.02638585e+06 -5.34876789e+05 -1.12116128e+04
  -3.12081715e+05  2.28182065e+06]
 [-6.58814826e+05 -1.67741421e+06  1.72015326e+06 -5.54890369e+05
   1.62778984e+06  1.47343528e+06  7

In [28]:
n=10

A = np.zeros((n, n))

B = make_sparse_spd_matrix()
min_eig = np.min(np.linalg.eigvals(B))
print(f"Minimum eigenvalue ({min_eig:.2e})")

C = -2*np.eye(n)

# symmetric D
D = make_sparse_spd_matrix()

# initial guess
X0 = np.eye(n)

X_hat = newton_nare(A, B, C, D, X0)

residual = C + X_hat @ A + D @ X_hat - X_hat @ B @ X_hat
residual_norm = norm(residual, 'fro')

print('Residual after NARE: ', residual)
print('Its norm: ', residual_norm)

Minimum eigenvalue (4.16e-01)
Iteration 0, Error: 1.28e+00
Iteration 5, Error: 1.04e+00
Iteration 10, Error: 2.04e+00
Iteration 15, Error: 9.95e-01
Iteration 20, Error: 1.03e+00
Iteration 25, Error: 1.08e+00
Residual after NARE:  [[-5.69176063e+00 -2.61139792e-11  0.00000000e+00  6.40863012e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.60881804e-11  0.00000000e+00]
 [ 1.64522594e-10 -1.78381057e+00  0.00000000e+00 -1.65913968e-10
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.78510715e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -6.02208382e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 6.40863012e-01 -1.81316795e-11  0.00000000e+00 -5.69176063e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.80799380e-11  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -6.02208382e+00  0.00000000e+00  0